# CamShift Tracking

In [7]:
import numpy as np
import cv2 

# Bắt video
cap = cv2.VideoCapture(0)

# Chụp khung hình đầu tiên từ video
ret, khung_hinh = cap.read()

# Thiết lập cửa sổ theo dõi ban đầu


# Trước hết, chúng ta sẽ phát hiện khuôn mặt và sử dụng nó làm hình chữ nhật ban đầu.
bo_loc_khuon_mat = cv2.CascadeClassifier('../DATA/haarcascades/haarcascade_frontalface_default.xml')
khung_khuon_mat = bo_loc_khuon_mat.detectMultiScale(khung_hinh) 

# Chuyển danh sách này thành một tuple (x, y, w, h)
(x_khuon_mat, y_khuon_mat, w_khuon_mat, h_khuon_mat) = tuple(khung_khuon_mat[0]) 
cua_so_theo_doi = (x_khuon_mat, y_khuon_mat, w_khuon_mat, h_khuon_mat)
# Thiết lập ROI để theo dõi
roi = khung_hinh[y_khuon_mat:y_khuon_mat+h_khuon_mat, x_khuon_mat:x_khuon_mat+w_khuon_mat]

# Sử dụng ánh xạ màu HSV
hsv_roi =  cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

# Tìm histogram để ánh xạ ngược mục tiêu trên mỗi khung hình để tính toán meanshit
roi_hist = cv2.calcHist([hsv_roi],[0],None,[180],[0,180])

# Chuẩn hóa giá trị mảng histogram với giá trị tối thiểu là 0 và tối đa là 255
cv2.normalize(roi_hist,roi_hist,0,255,cv2.NORM_MINMAX)

# Thiết lập điều kiện kết thúc, hoặc sau 10 lần lặp hoặc di chuyển ít nhất 1 điểm
term_crit = ( cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1 )

while True:
    ret ,khung_hinh = cap.read()
    if ret == True:
        
        # Chuyển đổi khung hình sang không gian màu HSV
        hsv = cv2.cvtColor(khung_hinh, cv2.COLOR_BGR2HSV)
        
        # Tính toán phản chiếu ngược dựa trên histogram_roi đã tạo trước đó
        dst = cv2.calcBackProject([hsv],[0],roi_hist,[0,180],1)
        
        #########################################################
        #########################################################
        ########## CAM SHIFT ####################################
        ########################################################
        #######################################################
        
        # Áp dụng Camshift để có được tọa độ mới của hình chữ nhật
        ret, cua_so_theo_doi = cv2.CamShift(dst, cua_so_theo_doi, term_crit)
        
        # Vẽ nó lên ảnh
        pts = cv2.boxPoints(ret)
        pts = np.int0(pts)
        img2 = cv2.polylines(khung_hinh,[pts],True, (0,0,255),5)
        cv2.imshow('img2',img2)
        
        ########################################################
        #######################################################
        ########################################################
        #######################################################
        
        k = cv2.waitKey(1) & 0xff
        if k == 27:
            break
        
    else:
        break
        
cv2.destroyAllWindows()
cap.release()


IndexError: tuple index out of range